In [1]:
# Autoreloading
%load_ext autoreload
%autoreload 2

In [12]:
# Imports
from cluster_client.manager import ClusterManager
from cluster_client.model.field_expressions import *
from cluster_client.model.aggregate_expressions import *
from cluster_client.upload.cassandra import *
from cluster_client.connector.cassandra import CassandraConnector
from cluster_client.test_data.generate_test_loan_data import loan_amortisation_generator, loan_amortisation_metadata, loan_origination_generator, loan_origination_metadata

In [ ]:
# Upload
connector = CassandraConnector("localhost", 9042)
upload_handler = CassandraUploadHandler(connector)
upload_handler.create_from_csv(r"D:\uni\Y4S1\Project\forex.csv", "test", "forex_table", ["slug"], ["date"])

In [ ]:
# Upload from iterator
connector = CassandraConnector("localhost", 9042)
upload_handler = CassandraUploadHandler(connector)
(column_names, column_types) = loan_amortisation_metadata()
upload_handler.drop_table("test", "amortisation_table").create_from_iterator(loan_amortisation_generator(100), "test", "amortisation_table", column_names, column_types, ["loan_id"])

In [3]:
# Setup cluster manager
CLUSTER_ADDRESS = "localhost"
manager = ClusterManager(CLUSTER_ADDRESS)

In [4]:
select = manager.cassandra_table("test", "forex_table").filter(F("slug") == V("GBP/INR")).evaluate()

In [20]:
group_by = manager.cassandra_table("test", "forex_table").group_by([F("slug")], [Avg(F("high"))]).evaluate()

In [21]:
result_df = group_by.get_dataframe()
print(result_df)

        slug      Avg_high
0    GBP/RUB     65.278543
1    USD/SDG     28.156853
2    JPY/USD      0.009414
3    GBP/MAD     13.454675
4    JPY/EUR      0.008064
..       ...           ...
184  EUR/IDR  13907.585927
185  GBP/BDT    113.329460
186  AUD/JPY     84.940844
187  GBP/MUR     49.469162
188  GBP/ISK    172.988541

[189 rows x 2 columns]
